In [1]:
import torch
from IPython import display
from d2l import torch as d2l

In [2]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

C:\Users\25832\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
next(iter(train_iter))

[tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.6824,  ..., 0.4706, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.8196,  ..., 0.7647, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0275, 0.0000, 0.0000]]],
 
 
         [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],
 
 
         [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000

In [4]:
next(iter(test_iter))[0].shape

torch.Size([256, 1, 28, 28])

In [5]:
num_inputs = 784  #(28*28)
num_ouputs = 10   #(class)

In [6]:
w = torch.normal(0,0.001,size=(num_inputs,num_ouputs),requires_grad=True)

In [7]:
b = torch.zeros(num_ouputs,requires_grad=True)

In [8]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1,keepdim=True)
    return X_exp/partition

In [9]:
X = torch.normal(0,1,(2,5))
X

tensor([[-0.3964,  0.3296, -0.2666,  1.1357,  1.1734],
        [-1.4112, -1.0940, -1.3599, -0.7049,  1.1375]])

In [10]:
X_prob = softmax(X)
X_prob

tensor([[0.0733, 0.1515, 0.0835, 0.3393, 0.3524],
        [0.0548, 0.0753, 0.0577, 0.1111, 0.7011]])

In [11]:
X_prob.sum(1,keepdim=True)

tensor([[1.],
        [1.]])

In [12]:
def net(X):
    return torch.matmul(X.reshape(-1,w.shape[0]),w) + b

In [13]:
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

tensor([0.1000, 0.5000])

In [14]:
def cross_entropy(y_hat,y):
    return -torch.log(y_hat[range(len(y_hat)),y])
cross_entropy(y_hat,y)

tensor([2.3026, 0.6931])

In [15]:
def accuracy(y_hat,y):
    if len(y_hat.shape)>1 and y_hat.shape[1]>1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype)==y
    return float(cmp.type(y.dtype).sum())

In [16]:
accuracy(y_hat,y)/len(y)

0.5

In [17]:
class Acumulator:
    def __init__(self,n):
        self.data = [0.0] * n
    def add(self,*args):
        self.data = [a+float(b) for a,b in zip(self.data,args)]
    def reset(self):
        self.data = [0.0] * len(self.data)
    def __getitem__(self,idx):
        return self.data[idx]


In [21]:
metric = Acumulator(2)

In [22]:
def evaluate_accuracy(net,data_iter):
    if isinstance(net,torch.nn.Module):
        net.eval()
    mertric = Acumulator(2)
    for X,y in data_iter:
        mertric.add(accuracy(net(X),y),y.numel())
    return mertric[0]/mertric[1]

In [25]:
def train(net,train_iter,loss,updater):
    if isinstance(net,torch.nn.Module):
        net.train()
    metric = Acumulator(3)
    for X,y in train_iter:
        y_hat = net(X)
        l = loss(y_hat,y)
        
        if isinstance(updater,torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            updater.step()
            metric.add(float(l)*len(y),accuracy(y_hat,y),y.size().numel())
        